Model Ideas:

Transfer Learning:
- Vision Transformer (https://github.com/google-research/vision_transformer)
- Vision Transformer Adapter (https://github.com/czczup/vit-adapter)
- EfficientNetV2 ()
- ConvNeXt
- ResNET50
- Novel arch using cosine similarity instead of convolution (see: https://github.com/Ritz-19/cosine_sim_NN)


Data pipeline:
- unzip datafiles
- 

In [2]:
conda install scikit-image

Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\Derek\AppData\Local\R-MINI~1\envs\tf-gpu-3

  added / updated specs:
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.0               |       h19a0ad4_0         139 KB
    brotli-1.0.9               |       ha925a31_2         332 KB
    bzip2-1.0.8                |       he774522_0         113 KB
    certifi-2022.5.18.1        |   py39haa95532_0         157 KB
    cfitsio-3.470              |       he774522_6         512 KB
    charls-2.2.0               |       h6c2663c_0          82 KB
    cloudpickle-2.0.0          |     pyhd3eb1b0_0          32 KB
    cytoolz-0.11.0             |   py39h2bbff1b_0         294 KB
    dask-core-2022.5.0         |   py39haa95532_0         1.6 MB
    fsspec-2022.3.0            |   py39haa95532_0         208 KB
    imagecodecs-2021.8



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
import numpy as np
import os.path
from PIL import Image
from glob import glob
from skimage.transform import resize

Read data files:

In [1]:
# training
training_x_path = './datasets/training/training/images/*.tif'
training_y_path = './datasets/training/training/1st_manual/*.gif'
# testing
test_x_path = './datasets/test/test/images/*.tif'
test_y_path = './datasets/test/test/1st_manual/*.gif'
test_mask_path = './datasets/test/test/mask/*.gif'

data_paths = [training_x_path, training_y_path, test_x_path, test_y_path, test_mask_path]

# shape
data_shape = (576, 576)

In [ ]:
def read_input(path):

    if path.find('2nd') > 0 and path.find('DRIVE') > 0:
        x = np.array(Image.open(path)) / 1.
    else:
        x = np.array(Image.open(path)) / 255.
    if x.shape[-1] == 3:
        return x
    else:
        return x[..., np.newaxis]


def preprocessData(data_path, dataset):
    
    global data_shape

    data_path = list(sorted(glob(data_path)))

    if data_path[0].find('mask') > 0:
        return np.array([read_input(image_path) for image_path in data_path])
    else:
        return np.array([resize(read_input(image_path), data_shape) for image_path in data_path])

Alternate method to read in data:

In [5]:
conda install -c conda-forge opencv

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\Derek\AppData\Local\R-MINI~1\envs\tf-gpu-3

  added / updated specs:
    - opencv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.5.18.1|       h5b45459_0         180 KB  conda-forge
    certifi-2022.5.18.1        |   py39hcbf5309_0         151 KB  conda-forge
    freeglut-3.2.2             |       h0e60522_1         150 KB  conda-forge
    jasper-2.0.33              |       h77af90b_0         677 KB  conda-forge
    libblas-3.9.0              |1_h8933c1f_netlib         193 KB  conda-forge
    libcblas-3.9.0             |5_hd5c7e75_netlib          95 KB  c



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [25]:
import tensorflow as tf
import pandas as pd
from utils.prepare_data import *
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from PIL import Image

print(os.listdir())
print(os.getcwd())

['.git', '.gitignore', 'datasets', 'modelExperiments.ipynb', 'README.md', 'utils']
c:\Users\Derek\Documents\SLU_Capstone


In [17]:
dir_path = os.getcwd()

# training
training_x_path = dir_path + '/datasets/training/training/images/'
training_y_path = dir_path + '/datasets/training/training/1st_manual/'
# testing
test_x_path = dir_path + '/datasets/test/test/images/'
test_y_path = dir_path + '/datasets/test/test/1st_manual/'
test_mask_path = dir_path + '/datasets/test/test/mask/'

dir_path = os.getcwd()
train_files = os.listdir(training_x_path)
train_data = []
train_label = []

In [18]:
desired_size = 592

for i in train_files:
    im = cv2.imread(training_x_path + i)
    label = cv2.imread(training_x_path + i.split('_')[0] + '_manual1.png', cv2.IMREAD_GRAYSCALE)

    old_size = im.shape[:2]  # old_size is in (height, width) format
    delta_w = desired_size - old_size[1]
    delta_h = desired_size - old_size[0]

    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [0, 0, 0]
    color2 = [0]

    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    new_label = cv2.copyMakeBorder(label, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color2)

    train_data.append(cv2.resize(new_im, (desired_size, desired_size)))

    temp = cv2.resize(new_label, (desired_size, desired_size))
    _, temp = cv2.threshold(temp, 127, 255, cv2.THRESH_BINARY)
    train_label.append(temp)


train_data = np.array(train_data)
train_label = np.array(train_label)


x_train = train_data.astype('float32') / 255.
y_train = train_label.astype('float32') / 255.

# Implementation of https://arxiv.org/abs/2004.03702 is used
x_rotated, y_rotated, x_flipped, y_flipped = img_augmentation(x_train, y_train)

x_train= np.concatenate([x_train, x_rotated,x_flipped])
y_train = np.concatenate([y_train, y_rotated,y_flipped])

x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.10, random_state = 101)

x_train = np.reshape(x_train, (len(x_train), desired_size, desired_size, 3))
y_train = np.reshape(y_train, (len(y_train), desired_size, desired_size, 1))

x_validate = np.reshape(x_validate, (len(x_validate), desired_size, desired_size, 3))
y_validate = np.reshape(y_validate, (len(y_validate), desired_size, desired_size, 1))

In [22]:
train_files[0]

'21_training.tif'